In [8]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree, DecisionTreeClassifier
from sklearn import metrics
import sklearn.preprocessing
import imodels
from imodels.util.data_util import DSET_CLASSIFICATION_KWARGS

print(list(DSET_CLASSIFICATION_KWARGS.keys()))
# installable with: `pip install imodels`
np.random.seed(13)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['pima_diabetes', 'sonar', 'heart', 'diabetes', 'breast_cancer_recurrence', 'breast_cancer_wisconsin', 'credit_g', 'juvenile', 'compas', 'fico', 'readmission', 'adult', 'csi_pecarn', 'iai_pecarn', 'tbi_pecarn']


In [9]:
apple = pd.read_csv('apple_quality.csv')
apple = apple.dropna()
apple = apple.replace({"good":1, "bad":0})
X = apple[['Juiciness', 'Size', "Ripeness", "Crunchiness", "Sweetness", "Weight", "Acidity"]]
y = apple['Quality']
feature_names = ['Juiciness', 'Size', "Ripeness", "Crunchiness", "Sweetness", "Weight", "Acidity"]

/var/folders/ht/dg47d7j9403_2cfbpk6nk1xr0000gn/T/ipykernel_64671/1662780913.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  apple = apple.replace({"good":1, "bad":0})


In [10]:
print("shapes", X.shape, y.shape, "nunique", np.unique(y).size)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.2
)

m = sklearn.preprocessing.StandardScaler()
X_train = m.fit_transform(X_train)
X_test = m.transform(X_test)

m = imodels.AutoInterpretableClassifier()
m.fit(X_train, y_train, cv=3)

print("best params", m.est_.best_params_)
print("best score", m.est_.best_score_)
print("best estimator", m.est_.best_estimator_)
print("best estimator params", m.est_.best_estimator_.get_params())

shapes (4000, 7) (4000,) nunique 2


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:988: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 977, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 344, in _score
    response_method = _check_response_method(estimator, self._response_method)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

best params {'est': RuleFitClassifier(), 'est__max_rules': 100, 'est__n_estimators': 20}
best score 0.8894721823392943
best estimator Pipeline(steps=[('est', RuleFitClassifier(max_rules=100, n_estimators=20))])
best estimator params {'memory': None, 'steps': [('est', RuleFitClassifier(max_rules=100, n_estimators=20))], 'verbose': False, 'est': RuleFitClassifier(max_rules=100, n_estimators=20), 'est__alpha': None, 'est__cv': True, 'est__exp_rand_tree_size': True, 'est__include_linear': True, 'est__lin_standardise': True, 'est__lin_trim_quantile': 0.025, 'est__max_rules': 100, 'est__memory_par': 0.01, 'est__n_estimators': 20, 'est__random_state': None, 'est__sample_fract': 'default', 'est__tree_generator': None, 'est__tree_size': 4}


In [11]:
df = pd.DataFrame(m.est_.cv_results_).sort_values("rank_test_score")
first_cols = ["rank_test_score", "mean_test_score", "std_test_score"]
df = df[first_cols + [c for c in df.columns if c not in first_cols]].round(3)
# remove std_ cols
df = df[[c for c in df.columns if "std_" not in c]]
df

,rank_test_score,mean_test_score,mean_fit_time,mean_score_time,param_est,param_est__max_leaf_nodes,param_est__max_rules,param_est__n_estimators,param_est__n_boosting_rounds,param_est__C,param_est__l1_ratio,param_est__alpha,params,split0_test_score,split1_test_score,split2_test_score
4,1,0.889,46.901,0.141,RuleFitClassifier(max_rules=30),NaN,100,20,NaN,NaN,NaN,NaN,"{'est': RuleFitClassifier(max_rules=30), 'est_...",0.867,0.905,0.897
10,2,0.835,0.150,0.003,FIGSClassifier(max_rules=12),NaN,10,NaN,NaN,NaN,NaN,NaN,"{'est': FIGSClassifier(max_rules=12), 'est__ma...",0.841,0.845,0.818
6,3,0.834,2.518,0.046,TreeGAMClassifier(),NaN,NaN,NaN,100,NaN,NaN,NaN,"{'est': TreeGAMClassifier(), 'est__n_boosting_...",0.831,0.845,0.825
12,4,0.821,0.005,0.001,"LogisticRegression(penalty='elasticnet', solve...",NaN,NaN,NaN,NaN,0.1,1,NaN,{'est': LogisticRegression(penalty='elasticnet...,0.807,0.846,0.811
11,5,0.821,0.005,0.001,"LogisticRegression(penalty='elasticnet', solve...",NaN,NaN,NaN,NaN,0.1,0.5,NaN,{'est': LogisticRegression(penalty='elasticnet...,0.806,0.846,0.812
14,6,0.821,0.005,0.001,"LogisticRegression(penalty='elasticnet', solve...",NaN,NaN,NaN,NaN,1,1,NaN,{'est': LogisticRegression(penalty='elasticnet...,0.806,0.845,0.813
13,7,0.821,0.005,0.001,"LogisticRegression(penalty='elasticnet', solve...",NaN,NaN,NaN,NaN,1,0.5,NaN,{'est': LogisticRegression(penalty='elasticnet...,0.806,0.845,0.813
16,8,0.821,0.005,0.001,"LogisticRegression(penalty='elasticnet', solve...",NaN,NaN,NaN,NaN,10,1,NaN,{'est': LogisticRegression(penalty='elasticnet...,0.806,0.845,0.813
15,9,0.821,0.005,0.001,"LogisticRegression(penalty='elasticnet', solve...",NaN,NaN,NaN,NaN,10,0.5,NaN,{'est': LogisticRegression(penalty='elasticnet...,0.806,0.845,0.813
3,10,0.816,1.191,0.013,RuleFitClassifier(max_rules=30),NaN,10,20,NaN,NaN,NaN,NaN,"{'est': RuleFitClassifier(max_rules=30), 'est_...",0.814,0.831,0.803


In [12]:
est = m.est_.best_estimator_.named_steps['est']
print(est)

> ------------------------------
> RuleFit:
> 	Predictions are made by summing the coefficients of each rule
> ------------------------------
                                                                rule  coef
                                                                  X0  0.28
                                                                  X1  1.08
                                                                  X2  0.39
                                                                  X3  0.11
                                                                  X4  0.38
                                                                  X5  0.32
                                                                  X6 -0.07
                                                       X5 > -1.24782 -0.14
                                                         X0 > -0.945  0.14
                                                       X4 <= 0.81554 -0.16
                                 

In [30]:
pd.DataFrame({'feature_names': feature_names, 'coef': est.coef_.flatten()}).round(3)

AttributeError: 'RuleFitClassifier' object has no attribute 'coef_'

This classifier seems to not have coefficients, according to the documentation found for RuleFitClassifier.